In [1]:
import h5py
import numpy as np
import pandas as pd
from sklearn.utils import resample
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../Shared')
import processing, Model, Visualization, Adaptation
from sklearn.model_selection import train_test_split
import modules


def Run_one_subject(F_name, total_sessions):
    base_path = 'D:/Data/Gait-EMG/Mypredict/'
    f = h5py.File(base_path+f"{F_name}", "r")

    key_lst = list(f['Day_1'].keys())

    Features = ['mav', 'var', 'zc', 'iemg', 'wl', 'wamp', 'mavs', 'rms', 'ssc', 'msq', 'v3', 'ld', 'dabs', 'mfl', 'mpr', 'mnf', 'psr', 'arc1', 'arc2', 'arc3', 'arc4', 'cc1', 'cc2', 'cc3', 'cca', 'dwtc1', 'dwtc2', 'dwtpc1', 'dwtpc2', 'dwtpc3']
    Sel_feat_idx = 4

    window_size, step_size = 200, 10

    X_lst, y_lst = [], []
    for trials in key_lst[1:total_sessions+1]:
        print("\n\n===================================", trials, "===================================\n")
        trial = f["Day_1"][trials]

        # 1.
        label, valid_idx = modules.label_extraction(trial)
        emg_keys = [k for k in trial.keys() if k.startswith("EMG_")]
        X_emg = modules.stack_signals(trial, emg_keys, valid_idx)

        # 2.
        balanced_idx = modules.Downsample_to_balance_class(label)
        X_emg_bal, y_bal = X_emg[balanced_idx], label[balanced_idx]

        #X_emg_bal, y_bal = random_downsample(X_emg_bal, y_bal, fraction=0.5)
        #print("After downsampling - EMG:", X_emg_bal.shape, "y:", y_bal.shape)

        num_channels = X_emg.shape[1]
        X_tmp, y_tmp = processing.sliding_window_with_labels(X=X_emg_bal, y=y_bal, window_size=window_size, step_size=step_size)  # (num_windows, win_len, ch)

        all_X, all_y = modules.get_X_y_WL_only(X_tmp, y_tmp)
        all_y = modules.y_change_to_int(all_y)
        all_X = all_X.reshape(-1, num_channels, 1)

        print("\t EMG:", all_X.shape, "y:", all_y.shape)
        X_lst.append(all_X)
        y_lst.append(all_y)

    final_X = np.concatenate(X_lst, axis=0)
    final_y = np.concatenate(y_lst, axis=0)
    print("final X:", final_X.shape, "y:", final_y.shape)

    return final_X, final_y


### Used only five trials for each session

In [3]:
X, y = [], []
fnames = [f'MP10{i}.hdf5' for i in range(1,10)] + ["MP110.hdf5"]

for i in range(len(fnames)):
    X_tmp, y_tmp = Run_one_subject(F_name=f"{fnames[i]}", total_sessions=5)
    X.append(X_tmp)
    y.append(y_tmp)



=================================== Trial_01 ===================================

Unique labels: [0.  1.  2.  3.  5.  6.  7.  8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8]
New labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
	 EMG: (4031, 16, 1) y: (4031,)


=================================== Trial_02 ===================================

Unique labels: [0.  1.  2.  3.  4.  5.  6.  7.  8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8]
New labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
	 EMG: (3021, 16, 1) y: (3021,)


=================================== Trial_03 ===================================

Unique labels: [0.  2.  3.  4.  5.  6.  7.  8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8]
New labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
	 EMG: (2981, 16, 1) y: (2981,)


=================================== Trial_04 ===================================

Unique labels: [0.  2.  3.  4.  5.  6.  7.  8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8]
New labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
	 EMG: (2831, 16,

# Inter-subject leave one out

In [6]:
#Training time: 22min
ACC_All = []
for i in range(len(fnames)):
    print(f"Target : {fnames[i]}")
    X_target, y_target = X[i], y[i]
    mask = y_target != 16   # Boolean mask: True if label is not 16 ====> 첫 번째 데이터만 16이 없음
    X_target, y_target = X_target[mask], y_target[mask]

    X_lst, y_lst, Sub_acc = [], [], []

    for j in range(len(fnames)):
        if i == j:
            continue
        else:
            mask = y[j] != 16   # Boolean mask: True if label is not 16 ====> 첫 번째 데이터만 16이 없음
            X_tmp, y_tmp = X[j][mask], y[j][mask]

            X_lst.append(X_tmp)
            y_lst.append(y_tmp)
            X_train, y_train = np.concatenate(X_lst, axis=0), np.concatenate(y_lst, axis=0)

            model = Model.build_model_1D(input_shape=X_train.shape[1:], num_classes=len(np.unique(y_train)))
            history = model.fit(X_train, y_train, epochs=50, batch_size=512, verbose=0)
            loss, acc = model.evaluate(X_target, y_target, verbose=0)
            print(f"\t Subject added : {fnames[j]} - X_train: {X_train.shape}, y_train: {y_train.shape}  ===> accuracy : {acc*100:.2f}")
            Sub_acc.append(acc)
    print("\n\n")
    ACC_All.append(Sub_acc)

Target : MP101.hdf5
	 Subject added : MP102.hdf5 - X_train: (15155, 16, 1), y_train: (15155,)  ===> accuracy : 10.63
	 Subject added : MP103.hdf5 - X_train: (29670, 16, 1), y_train: (29670,)  ===> accuracy : 7.53
	 Subject added : MP104.hdf5 - X_train: (44505, 16, 1), y_train: (44505,)  ===> accuracy : 8.84
	 Subject added : MP105.hdf5 - X_train: (57260, 16, 1), y_train: (57260,)  ===> accuracy : 10.11
	 Subject added : MP106.hdf5 - X_train: (72095, 16, 1), y_train: (72095,)  ===> accuracy : 10.48
	 Subject added : MP107.hdf5 - X_train: (88210, 16, 1), y_train: (88210,)  ===> accuracy : 10.32
	 Subject added : MP108.hdf5 - X_train: (102995, 16, 1), y_train: (102995,)  ===> accuracy : 14.92
	 Subject added : MP109.hdf5 - X_train: (116230, 16, 1), y_train: (116230,)  ===> accuracy : 10.11
	 Subject added : MP110.hdf5 - X_train: (132985, 16, 1), y_train: (132985,)  ===> accuracy : 14.00



Target : MP102.hdf5
	 Subject added : MP101.hdf5 - X_train: (15245, 16, 1), y_train: (15245,)  ===> 

In [7]:
ACC_All

[[0.10632994771003723,
  0.07530337572097778,
  0.08842243254184723,
  0.10114791989326477,
  0.10475565493106842,
  0.10324696451425552,
  0.1492292582988739,
  0.10114791989326477,
  0.13998031616210938],
 [0.15704387426376343,
  0.13309139013290405,
  0.19993402063846588,
  0.24790498614311218,
  0.25146815180778503,
  0.27931374311447144,
  0.19835038483142853,
  0.1634444147348404,
  0.14477069675922394],
 [0.08432655781507492,
  0.09094040840864182,
  0.17340682446956635,
  0.15266965329647064,
  0.13703066110610962,
  0.1466069519519806,
  0.12380295991897583,
  0.11353772133588791,
  0.17292456328868866],
 [0.08001348376274109,
  0.1537580043077469,
  0.1507246345281601,
  0.25493764877319336,
  0.18483316898345947,
  0.12874957919120789,
  0.1556454300880432,
  0.2619481086730957,
  0.2440849393606186],
 [0.13986672461032867,
  0.1342218667268753,
  0.15350842475891113,
  0.16417090594768524,
  0.20995688438415527,
  0.20235201716423035,
  0.19372795522212982,
  0.189102306962

In [8]:
pd.DataFrame(np.array(ACC_All).T, columns=fnames).to_csv("./leave_one_out_inter_subject_cumulative_fashion.csv", index=False)

# Few shot adaptation

In [ ]:
def run_intersub_leave_one_out(K):
    #Training time:
    ACC_all_before, ACC_all_after = [], []
    for i in range(len(fnames)):
        print(f"Current : {fnames[i]}")
        X_train, y_train = X[i], y[i]

        mask = y_train != 16   # Boolean mask: True if label is not 16 ====> 첫 번째 데이터만 16이 없음
        X_train = X_train[mask]
        y_train = y_train[mask]

        Sub_acc_before, Sub_acc_after = [], []

        for j in range(len(fnames)):
            if i == j:
                continue
            else:
                model = Model.build_model_1D(input_shape=X_train.shape[1:], num_classes=len(np.unique(y_train)))
                history = model.fit(X_train, y_train, epochs=50, batch_size=512, verbose=0)
                meta = Adaptation.MetaLearner(input_model=model, N_way=len(np.unique(y_train)), input_shape=X_train.shape[1:],
                        meta_iters=5,                             # Number of meta-training loops
                        meta_step_size=0.02                          # Reptile meta step
                    )
                X_filtered, y_filtered = X[j], y[j]
                mask = y_filtered != 16   # Boolean mask: True if label is not 16
                X_filtered = X_filtered[mask]
                y_filtered = y_filtered[mask]

                X_train_meta, X_test, y_train_meta, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)
                loss, acc_before = model.evaluate(X_test, y_test, verbose=0)
                acc_after = meta.train(X_train_meta, y_train_meta, X_test, y_test, Adaptation.get_data_Meta, N_way=len(np.unique(y_train)), K_shot=K)

                print(f"\t Target sub {fnames[j]} acc => before: {acc_before*100:.2f}% | after: {acc_after*100:.2f}%")
                Sub_acc_before.append(acc_before)
                Sub_acc_after.append(acc_after)

        print("\n\n")
        ACC_all_before.append(Sub_acc_before)
        ACC_all_after.append(Sub_acc_after)

    pd.DataFrame(np.array(ACC_all_before).T, columns=fnames).to_csv(f'./Result_K/Leave_one_subject_out_few_shot_K{K}_before.csv', index=False)
    pd.DataFrame(np.array(ACC_all_after).T, columns=fnames).to_csv(f'./Result_K/Leave_one_subject_out_few_shot_K{K}_after.csv', index=False)

In [ ]:
run_intersub_leave_one_out(K=1)
run_intersub_leave_one_out(K=2)
run_intersub_leave_one_out(K=5)
run_intersub_leave_one_out(K=10)
run_intersub_leave_one_out(K=20)